In [1]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)
google_drive_prefix = "/content/drive/My Drive"

Mounted at /content/drive/


In [2]:
%pwd

'/content'

In [3]:
import os
os.chdir('drive/My Drive')

In [4]:
%pwd

'/content/drive/My Drive'

In [5]:
!ls

'Colab Notebooks'   ITR-23   Text_Summarization  'Untitled Jam.gjam'


In [ ]:
!git clone https://github.com/OmkarBorhade98/Text_Summarization

Cloning into 'Text_Summarization'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 159 (delta 66), reused 128 (delta 38), pack-reused 0
Receiving objects: 100% (159/159), 39.46 KiB | 1.36 MiB/s, done.
Resolving deltas: 100% (66/66), done.


In [6]:
os.chdir('Text_Summarization')

In [7]:
! pip install -e .

Obtaining file:///content/drive/MyDrive/Text_Summarization
  Preparing metadata (setup.py) ... done
  Running setup.py develop for textSummarizer


In [ ]:
! pip install ensure

In [8]:
! pip install datasets[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00


In [9]:
! pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.0 MB/s eta 0:00:00


In [10]:
! pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 7.4 MB/s eta 0:00:00


In [ ]:
! git pull

remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 8 (delta 6), reused 8 (delta 6), pack-reused 0
Unpacking objects: 100% (8/8), 580 bytes | 6.00 KiB/s, done.
From https://github.com/OmkarBorhade98/Text_Summarization
   594a7db..ef53447  main       -> origin/main
Updating 594a7db..ef53447
Fast-forward
 research/04_model_trainer.ipynb                | 2 +-
 src/textSummarizer/components/model_trainer.py | 2 +-
 2 files changed, 2 insertions(+), 2 deletions(-)


In [ ]:
! python main.py

[2024-01-12 17:22:43,508 : INFO : utils : NumExpr defaulting to 2 threads.]
[2024-01-12 17:22:43,774 : INFO : config : PyTorch version 2.1.0+cu121 available.]
[2024-01-12 17:22:43,775 : INFO : config : TensorFlow version 2.15.0 available.]
[2024-01-12 17:22:43,777 : INFO : config : JAX version 0.4.23 available.]
2024-01-12 17:22:45.500766: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-12 17:22:45.500827: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-12 17:22:45.502873: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-12 17:22:47.262945: W tensorflow/compiler/t

In [11]:
from transformers import AutoTokenizer, AutoModel

In [12]:
tokenizer = AutoTokenizer.from_pretrained('artifacts/model_trainer/tokenizer')

In [13]:
from transformers import AutoModelForSeq2SeqLM
from datasets import load_dataset, load_from_disk, load_metric

In [14]:
model = AutoModelForSeq2SeqLM.from_pretrained('artifacts/model_trainer/model').to('cuda')

In [15]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=48d3757a697514df91bb40455d84e864521dbfb9cb82441f841ab7c162dbb164
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [16]:
dataset_samsum_pt = load_from_disk('artifacts/data_transformation/samsum_dataset/test')

rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

rouge_metric = load_metric('rouge')

<ipython-input-16-c829635eaeeb>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [18]:
# Evaluation
device = 'cuda'
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [20]:
from tqdm import tqdm
import pandas as pd

In [21]:
score = calculate_metric_on_test_ds(
    dataset_samsum_pt[0:10], rouge_metric, model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)


100%|██████████| 5/5 [00:11<00:00,  2.32s/it]


In [22]:
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.022834,0.0,0.022455,0.022312


In [23]:
dialogue = """
Alex: I wanna paly video game!
Mom: I wont Let you!! Go study First!
Alex: But studying is for people who arent intelligent enough.
Mom: Exactly, That is why you need to study.
"""

In [35]:
tokens = tokenizer(dialogue, return_tensors="pt", max_length=1024, truncation=True).to('cuda')
print(tokens)

{'input_ids': tensor([[ 5296,   151,   125, 14243, 12955,   445,   545,   389,   147,  7069,
           151,   125, 14797,  1593,   119,  1242,  1859,   692,  1485,   147,
          5296,   151,   343,  4060,   117,   118,   200,   170,  1675,   144,
          6756,   505,   107,  7069,   151, 26229,   108,   485,   117,   447,
           119,   217,   112,   692,   107,     1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


In [37]:
summary_ids = model.generate(tokens["input_ids"], max_length=50, num_beams=8, length_penalty=2.0, early_stopping=True)
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(output)

Alex wants to play video game. Mom won't let him play it. He needs to study first, because he's not intelligent enough to play it.
